In [1]:
import pandas as pd

In [9]:
df = pd.read_csv("part-00000", sep = '\t', header = None, names = ['key', 'debtincome'])

In [4]:
df.head()

,key,debtincome
0,"('Alabama', 'Anniston, Oxford - AL', 'Calhoun ...",1.731879
1,"('Alabama', 'Anniston, Oxford - AL', 'Calhoun ...",1.786440
2,"('Alabama', 'Anniston, Oxford - AL', 'Calhoun ...",1.838941
3,"('Alabama', 'Anniston, Oxford - AL', 'Calhoun ...",1.944490
4,"('Alabama', 'Anniston, Oxford - AL', 'Calhoun ...",1.946002


In [36]:
df.loc[:,'state'] = df['key'].apply(lambda x: eval(x)[0])
df.loc[:,'msa'] = df['key'].apply(lambda x: eval(x)[1])
df.loc[:,'county'] = df['key'].apply(lambda x: eval(x)[2])
df.loc[:,'year'] = df['key'].apply(lambda x: eval(x)[3])

In [43]:
df = df[['state', 'msa', 'county', 'year', 'debtincome']]

In [47]:
df.sort(['year', 'debtincome'], ascending = [True, False], inplace = True)

In [59]:
df2010 = df[(df['year'] == '2010') & (df['state'] != 'Puerto Rico')]

In [61]:
df2010['debtincome'].describe()

count    1100.000000
mean        1.936381
std         0.440259
min         0.524301
25%         1.668230
50%         1.921164
75%         2.220707
max         5.771094
Name: debtincome, dtype: float64